In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-12 16:07:45--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.42MB/s    in 0.2s    

2022-08-12 16:07:46 (5.42 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

In [5]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])

In [7]:
#Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
#load in a sql function to use columns
from pyspark.sql.functions import col

cleaned_df = vine_df.filter(col("total_votes") >= 20)
cleaned_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|
|R13VL62Y2HBQ0B|          5|           15|         21|   N|                Y|
|R22G55KAPZKJQV|          4|           20|         21|   N|                Y|
|R1610PGTJS7G3N|          2|           28|         44|   N|                Y|
| RLQL04BL0QXOJ|          4|           45|         47|   N|                Y|
|R2AYJHH8WJNGAU|          4|           98|        104|   N|                Y|
|R111DJA10Y6CMU|          5|           26|         39|   N|                Y|
|R2EE2TR4MRDV0U|          5|           53|         59|   N|                Y|
| RD4A80I5JDHED|          5|           69|         77|   N|                N|
|R1GU6IYZQWQE8X|          2|           24|         25|   N|     

In [8]:
#create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
helpful_df = cleaned_df.filter(col("helpful_votes")/col("total_votes") >= .5)
helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|
|R13VL62Y2HBQ0B|          5|           15|         21|   N|                Y|
|R22G55KAPZKJQV|          4|           20|         21|   N|                Y|
|R1610PGTJS7G3N|          2|           28|         44|   N|                Y|
| RLQL04BL0QXOJ|          4|           45|         47|   N|                Y|
|R2AYJHH8WJNGAU|          4|           98|        104|   N|                Y|
|R111DJA10Y6CMU|          5|           26|         39|   N|                Y|
|R2EE2TR4MRDV0U|          5|           53|         59|   N|                Y|
| RD4A80I5JDHED|          5|           69|         77|   N|                N|
|R1GU6IYZQWQE8X|          2|           24|         25|   N|     

In [9]:
#create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
yes_df = helpful_df.filter(col("vine") == "Y")
yes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1MAOLI5FJHAFM|          4|          249|        261|   Y|                N|
| R9PYAUDIBJVEC|          4|           12|         22|   Y|                N|
| R6V9SHMMG5M8F|          5|          101|        110|   Y|                N|
|R37PVLT6ELL5J4|          4|          181|        209|   Y|                N|
| R2FSFGWZF24V9|          4|           50|         51|   Y|                N|
|R3SRW1E8J56IGV|          5|          262|        281|   Y|                N|
| R86Z11D4CWOFM|          4|           32|         36|   Y|                N|
| RNP01HW9YISJO|          4|           20|         23|   Y|                N|
|R3KLACA6LCDZ0S|          3|           21|         22|   Y|                N|
| RZEQYOT2RE0N7|          4|           75|         80|   Y|     

In [10]:
#retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
no_df = helpful_df.filter(col("vine") == "N")
no_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|
|R13VL62Y2HBQ0B|          5|           15|         21|   N|                Y|
|R22G55KAPZKJQV|          4|           20|         21|   N|                Y|
|R1610PGTJS7G3N|          2|           28|         44|   N|                Y|
| RLQL04BL0QXOJ|          4|           45|         47|   N|                Y|
|R2AYJHH8WJNGAU|          4|           98|        104|   N|                Y|
|R111DJA10Y6CMU|          5|           26|         39|   N|                Y|
|R2EE2TR4MRDV0U|          5|           53|         59|   N|                Y|
| RD4A80I5JDHED|          5|           69|         77|   N|                N|
|R1GU6IYZQWQE8X|          2|           24|         25|   N|     

In [37]:
#Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid)

yes_total_review = yes_df.select("review_id").distinct().count()
yes_five_star = yes_df.filter(col("star_rating") == 5).count()
yes_percent_five_star = (yes_five_star/yes_total_review) * 100

no_total_review = no_df.select("review_id").distinct().count()
no_five_star = no_df.filter(col("star_rating") == 5).count()
no_percent_five_star = (no_five_star/no_total_review) * 100

